<a href="https://colab.research.google.com/github/danbernstein/parkingdirty/blob/master/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1
**Object Detection API configuration**: In this step, the model for object detection is downloaded, also some copying and deletion of references are done to leave the whole scheme configured.

In [0]:
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

# Step 2

**Imports** needed to run the Object Detection API demonstration

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt

from PIL import Image

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Step 3
**Configuration** of the model to use, path to the frozen inference graph and extra config elements for the Object detection API implementation.

In [0]:
# What model to download.
# MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'

# model with more accurancy but up to you use a diferent model
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_2017_11_08'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Step 4
Section with demo images enviroment

In [0]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'object_detection/test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '2016-09-16 154115 cam135.png'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

# Step 5
piece of code that represent the concrete detection, calling the TF session

In [0]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    sess.run(tf.global_variables_initializer())
    img = 1
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      

In [37]:
# Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          min_score_thresh=0.4,
          use_normalized_coordinates=True,
          line_thickness=8)
      
      plt.plot(lane, color = 'red')

      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)
      
      plt.imsave('object_detection/test_images/' + 'out' + '.jpg', image_np)
      img += 1

      
      scores = np.squeeze(scores)
      boxes = np.squeeze(boxes)
      # Return found objects
      print([category_index.get(i) for i in classes[0]])
      print(boxes.shape)
      print(num)


[{'id': 3, 'name': 'car'}, {'id': 8, 'name': 'truck'}, {'id': 3, 'name': 'car'}, {'id': 8, 'name': 'truck'}, {'id': 6, 'name': 'bus'}, {'id': 6, 'name': 'bus'}, {'id': 28, 'name': 'umbrella'}, {'id': 8, 'name': 'truck'}, {'id': 3, 'name': 'car'}, {'id': 6, 'name': 'bus'}, {'id': 8, 'name': 'truck'}, {'id': 1, 'name': 'person'}, {'id': 3, 'name': 'car'}, {'id': 8, 'name': 'truck'}, {'id': 3, 'name': 'car'}, {'id': 8, 'name': 'truck'}, {'id': 8, 'name': 'truck'}, {'id': 6, 'name': 'bus'}, {'id': 3, 'name': 'car'}, {'id': 1, 'name': 'person'}, {'id': 1, 'name': 'person'}, {'id': 3, 'name': 'car'}, {'id': 7, 'name': 'train'}, {'id': 8, 'name': 'truck'}, {'id': 3, 'name': 'car'}, {'id': 6, 'name': 'bus'}, {'id': 1, 'name': 'person'}, {'id': 6, 'name': 'bus'}, {'id': 3, 'name': 'car'}, {'id': 6, 'name': 'bus'}, {'id': 1, 'name': 'person'}, {'id': 3, 'name': 'car'}, {'id': 1, 'name': 'person'}, {'id': 1, 'name': 'person'}, {'id': 1, 'name': 'person'}, {'id': 3, 'name': 'car'}, {'id': 1, 'name

In [44]:
  num_cars_in_bikelane, num_cars_in_bus_stop, num_bikes_in_bike_lane, num_trucks_in_bus_stop = 0, 0, 0, 0
  import matplotlib.path as mpltPath

  lane = np.array([[156,284],[110,286],[257,154],[274,156]])
  pathrightlane = mpltPath.Path(lane)

# analyzing the detected objects for which are in the bikelane and converting into a tabular format 
  
  for i in range(boxes.shape[0]):
     if scores[i] > .4:
        box = tuple(boxes[i].tolist())
        #print(box)
        
        ymin, xmin, ymax, xmax = box

        center_x = (((xmax * 352) - (xmin * 352)) / 2) + (xmin * 352)
        center_y = (((ymax * 288) - (ymin * 288)) / 2) + (ymin * 288)
        points = [(center_x, center_y)]

        
        classes_int = np.squeeze(classes).astype(np.int32)

        if classes_int[i] in category_index.keys():
                            class_name = category_index[classes_int[i]]['name']  
          
        #print(class_name)
        if class_name in {'car', 'truck', 'bus', 'motorcycle','person'}:
          if pathrightlane.contains_points(points):
              num_cars_in_bikelane += 1
        if class_name == 'bicycle':
          points = [(center_x, center_y)]
          if pathrightlane.contains_points(points):
              num_bikes_in_bike_lane += 1      
         
  print(num_cars_in_bikelane, num_bikes_in_bike_lane)
                        

3 0


In [0]:
# get center of each box detection and align with the bike_lane status as coded by humans to see how it lines up

In [0]:
#calculate total time of blockage

# Custom Personal Images 

In [0]:
!mkdir images

In [0]:
!wget http://farm6.staticflickr.com/5485/9424913933_2d24e654ac_z.jpg -O images/custom_img_2.jpg

In [0]:
!wget http://farm4.staticflickr.com/3674/9412766091_03a2e22443_z.jpg -O images/custom_img_1.jpg

In [0]:
!ls ./images/

In [0]:
PATH_TO_TEST_IMAGES_DIR = 'images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'custom_img_{}.jpg'.format(i)) for i in range(1, 3) ]

IMAGE_SIZE = (12, 8)

In [0]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      image_np = load_image_into_numpy_array(image)
      image_np_expanded = np.expand_dims(image_np, axis=0)
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)